In [14]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import glob

class MahjongDataset(Dataset):
    def __init__(self, data_paths):
        self.x_dict = {
            "x_hand": None, 
            "x_hand_red": None, 
            "x_river": None, 
            "x_river_red": None, 
            "x_river_riichi": None, 
            "x_meld": None, 
            "x_meld_red5": None, 
            "x_meld_throw": None, 
            "x_meld_throw_red": None, 
            "x_dora": None, 
            "x_score": None, 
            "x_pool": None, 
            "x_winds": None}
        self.y = None
        self.loads = list(map(np.load, data_paths))
        self.loads = [np.load(path) for path in data_paths]
        
        self.load_index = -1
        self.load_next()

    def unload(self):
        self.y = None
        for key in self.x_dict.keys():
            self.x_dict[key] = None
    
    def load_next(self):
        self.load_index += 1
        self.load_index %= len(self.loads)
        self.y = self.loads[self.load_index]["y"][:2000].astype(np.float32)
        for key in self.x_dict.keys():
            self.x_dict[key] = self.loads[self.load_index][key][:2000].astype(np.float32)
        
        # Data pre-processing
        # Divide by 250 to average out the score values
        self.y /= 250
        self.x_dict["x_score"] /= 250
        self.x_dict["x_pool"] /= 250
        # Resize all input to B, X, 24, 9
        self.x_dict["x_hand"] = np.repeat(self.x_dict["x_hand"], 6, -2)
        self.x_dict["x_hand_red"] = np.expand_dims(self.x_dict["x_hand_red"], (-2, -3))
        self.x_dict["x_hand_red"] = np.repeat(self.x_dict["x_hand_red"], 3, -1)
        self.x_dict["x_hand_red"] = np.repeat(self.x_dict["x_hand_red"], 24, -2)
        self.x_dict["x_river"] = np.reshape(self.x_dict["x_river"], (*self.x_dict["x_river"].shape[:1], -1, *self.x_dict["x_river"].shape[3:]))
        self.x_dict["x_river_red"] = np.expand_dims(self.x_dict["x_river_red"], -1)
        self.x_dict["x_river_red"] = np.repeat(self.x_dict["x_river_red"], 9, -1)
        self.x_dict["x_river_riichi"] = np.expand_dims(self.x_dict["x_river_riichi"], -1)
        self.x_dict["x_river_riichi"] = np.repeat(self.x_dict["x_river_riichi"], 9, -1)
        self.x_dict["x_meld"] = np.reshape(self.x_dict["x_meld"], (*self.x_dict["x_meld"].shape[:1], -1, *self.x_dict["x_meld"].shape[3:]))
        self.x_dict["x_meld_red5"] = np.expand_dims(self.x_dict["x_meld_red5"], -2)
        # self.x_dict["x_meld_red5"] = np.repeat(self.x_dict["x_meld_red5"], 3, -1)
        # self.x_dict["x_meld_red5"] = np.repeat(self.x_dict["x_meld_red5"], 24, -2)
        
        for varname in self.x_dict.keys():
            print(varname, self.x_dict[varname].shape)
        # To predict score difference at the end of the round
        self.y -= self.x_dict["x_score"]  
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        x = {key: self.x_dict[key][idx] for key in self.x_dict.keys()}
        return x, self.y

# train_ds = MahjongDataset(sorted(glob.glob("dataset/riichi_ds_v02/*.npz"))[:30])
test_ds = MahjongDataset(sorted(glob.glob("dataset/riichi_ds_v02/*.npz"))[30:])

x_hand (2000, 4, 24, 9)
x_hand_red (2000, 1, 24, 9)
x_river (2000, 16, 24, 9)
x_river_red (2000, 4, 24, 9)
x_river_riichi (2000, 4, 24, 9)
x_meld (2000, 16, 4, 9)
x_meld_red5 (2000, 4, 1, 3)
x_meld_throw (2000, 4, 4, 4, 9)
x_meld_throw_red (2000, 4, 3)
x_dora (2000, 4, 4, 9)
x_score (2000, 4)
x_pool (2000,)
x_winds (2000, 2, 9)


In [24]:
test_ds[:8][0]["x_hand_red"].shape

(8, 1, 1, 3)

In [20]:
test_ds[:4][0]["x_hand"]

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 1., 1.],
         [0., 0., 0., ..., 0., 1., 1.],
         [0., 0., 0., ..., 0., 1., 1.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 1., ..., 1., 1., 0.],
         [0., 0., 1., ..., 1., 1., 0.],
         [0., 0., 1., ..., 1., 1., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 

In [12]:
len(test_ds)

262763

In [13]:
test_ds.load_next()

(270608, 4, 4, 9)


In [14]:
len(test_ds)

270608